In [1]:
from detectron2.modeling import build_model

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataclasses import dataclass
from detectron2.config import get_cfg
from config import get_settings
from ai.ditod import add_vit_config
@dataclass
class Arguments:
    dataset = "doclaynet"
    config_file_path = "ai/Configs/doclaynet_VGT_cascade_PTM.yaml"
    model_weights_path = "./models/model_final.pth"


args = Arguments()
cfg = get_cfg()
add_vit_config(cfg)
cfg.merge_from_file(args.config_file_path)


In [3]:
model = build_model(cfg)

use_pretrain_weight: load model from: ./models/layoutlm/


In [4]:
from detectron2.structures import ImageList, Instances
import torch
def preprocess_image(images):
    """
    Normalize, pad and batch the input images.
    """
    pixel_mean, pixel_std = torch.tensor([103.530, 116.280, 123.675]), torch.tensor([57.375, 57.120, 58.395])
    # images = [x["image"].to('cpu') for x in batched_inputs]
    images = [(x.permute(2,1,0) - pixel_mean) / pixel_std for x in images]
    images = [x.permute(2,1,0) for x in images]
    images = ImageList.from_tensors(
        images,
        0,
        padding_constraints={},
    )
    print(images.tensor.shape)
    return images

In [5]:
import torch 
import numpy as np
input_tensor = torch.rand(3,1000,1000)
input_dict = [{
    'input_ids':[2,444,566],
    'bbox': np.array([[112,334,444,666], [23,35,666,435], [22,66,343,432]]),
    'height': 1000,
    'width': 1000
}]
image_batch= preprocess_image([input_tensor])

torch.Size([1, 3, 1000, 1000])


In [6]:
model = model.eval().to('cpu')

In [3]:
import torch
quant =torch.ao.quantization.QuantStub()
dequant = torch.ao.quantization.DeQuantStub()
x = torch.rand(2,2)
print(x)
quant(x)

tensor([[0.3068, 0.9300],
        [0.0045, 0.7737]])


tensor([[0.3068, 0.9300],
        [0.0045, 0.7737]])

In [4]:
onnx_program = torch.onnx.dynamo_export(model, (image_batch, input_dict))

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [13]:
from torch.onnx import OperatorExportTypes
import onnx
import io
def export_onnx_model(model, inputs):
    """
    Trace and export a model to onnx format.

    Args:
        model (nn.Module):
        inputs (tuple[args]): the model will be called by `model(*inputs)`

    Returns:
        an onnx model
    """
    assert isinstance(model, torch.nn.Module)

    # make sure all modules are in eval mode, onnx may change the training state
    # of the module if the states are not consistent
    def _check_eval(module):
        assert not module.training

    model.apply(_check_eval)

    # Export the model to ONNX
    with torch.no_grad():
        with io.BytesIO() as f:
            torch.onnx.export(
                model,
                inputs,
                f,
                operator_export_type=OperatorExportTypes.ONNX_ATEN_FALLBACK,
                # verbose=True,  # NOTE: uncomment this for debugging
                # export_params=True,
                input_names=['image_batch', 'batched_inputs']
            )
            onnx_model = onnx.load_from_string(f.getvalue())

    return onnx_model

In [14]:
export_onnx_model(model, (image_batch, input_dict))

RuntimeError: Only tuples, lists and Variables are supported as JIT inputs/outputs. Dictionaries and strings are also accepted, but their usage is not recommended. Here, received an input of unsupported type: ImageList

In [14]:
import torch
# Simple module for demonstration
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.param = torch.nn.Parameter(torch.rand(3, 4))
        self.linear = torch.nn.Linear(4, 5)

    def forward(self, x):
        return self.linear(x + self.param).clamp(min=0.0, max=1.0)

module = MyModule()

In [21]:
inp = torch.rand(3,4)

In [22]:
import torch.onnx as onnx
onnx.export(module, args=(inp), f="aba.onnx")

In [9]:
quantized = torch.ao.quantization.quantize_dynamic(model ,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8) 

In [10]:
quantized(image_batch, input_dict)

[{'input_ids': [2, 444, 566], 'bbox': array([[112, 334, 444, 666],
       [ 23,  35, 666, 435],
       [ 22,  66, 343, 432]]), 'height': 1000, 'width': 1000}]
tensor([[[[-1.7898, -1.8010, -1.7919,  ..., -1.7980, -1.7927, -1.7878],
          [-1.7907, -1.8043, -1.7952,  ..., -1.8007, -1.7951, -1.8032],
          [-1.7870, -1.8006, -1.7966,  ..., -1.8015, -1.7943, -1.8031],
          ...,
          [-1.7905, -1.7896, -1.8011,  ..., -1.7940, -1.7958, -1.7947],
          [-1.8039, -1.7917, -1.7990,  ..., -1.7994, -1.7912, -1.7949],
          [-1.8007, -1.8010, -1.7872,  ..., -1.7958, -1.7924, -1.7906]],

         [[-2.0244, -2.0194, -2.0351,  ..., -2.0293, -2.0264, -2.0265],
          [-2.0332, -2.0199, -2.0200,  ..., -2.0330, -2.0197, -2.0330],
          [-2.0203, -2.0304, -2.0258,  ..., -2.0213, -2.0356, -2.0346],
          ...,
          [-2.0319, -2.0270, -2.0317,  ..., -2.0262, -2.0291, -2.0218],
          [-2.0239, -2.0288, -2.0203,  ..., -2.0231, -2.0307, -2.0311],
          [-2.024

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TypeError: linear(): argument 'weight' must be Tensor, not method

In [11]:
import torch
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
import copy


#
# post training dynamic/weight_only quantization
#

# we need to deepcopy if we still want to keep model_fp unchanged after quantization since quantization apis change the input model
model_to_quantize = copy.deepcopy(model)
model_to_quantize.eval()
qconfig_mapping = QConfigMapping().set_global(torch.ao.quantization.default_dynamic_qconfig)
# a tuple of one or more example inputs are needed to trace the model
# example_inputs = (torch.rand(3,224,224))
# prepare
example_input = (image_batch, input_dict,)
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, {"": example_input})
# no calibration needed when we only have dynamic/weight_only quantization
# quantize
model_quantized = quantize_fx.convert_fx(model_prepared)

Proxy(batched_inputs)
Proxy(getattr_1)
Proxy(size_1)


TypeError: zeros() received an invalid combination of arguments - got (tuple, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
a = list(list(model.children())[0].children())[0]

In [43]:
a

Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))

In [5]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
model = fasterrcnn_resnet50_fpn(pretrained=True)
dummpy_input = torch.randn(1, 3, 1333, 800)
model.eval()
model(dummpy_input)

# Export the model
torch.onnx.export(model,               # model being run
                  dummpy_input,                         # model input (or a tuple for multiple inputs)
                  "model.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],)

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/saeed/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:11<00:00, 14.1MB/s] 
/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torch/nn/functional.py